In [1]:
# page 79

!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 9.3 MB/s eta 0:00:00


In [2]:
# page 80

%%writefile word_count.py
from mrjob.job import MRJob

class MRWordCount(MRJob):
  def mapper(self, _, line):
    for word in line.split():
      yield(word, 1)

  def reducer(self, word, counts):
    yield(word, sum(counts))

if __name__ == '__main__':
  MRWordCount.run()

Writing word_count.py


In [3]:
!python word_count.py input.txt > output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.root.20250605.020017.907560
Running step 1 of 1...
job output is in /tmp/word_count.root.20250605.020017.907560/output
Streaming final output from /tmp/word_count.root.20250605.020017.907560/output...
Removing temp directory /tmp/word_count.root.20250605.020017.907560...


In [43]:
# page 87

%%writefile top_salary.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

cols="lastName,middleInitial,firstName,jobClass,agencyName,agencyID,annualSalary,grossPay,hireDate,fiscalYear,ObjectId".split(",")

class salarymax(MRJob):
  def mapper(self, _, line):
    # 각 줄을 딕셔너리로 변환
    # AttributeError: '_csv.reader' object has no attribute 'next'
    #row = dict(zip(cols, [a for a in csv.reader([line])]))

    reader = csv.reader([line])
    values = next(reader)
    row = dict(zip(cols, values))

    #self.stdout.write((f"\n\nRow being processed: {row}\n\n").encode('utf-8'))

    # 급여를 생성합니다
    try:
      yield 'salary', (float(row['annualSalary'][1:]), line)
    except ValueError:
      self.increment_counter('warn','missing salary', 1)
      yield 'salary', 0

    # 총 급여를 생성합니다
    try:
      yield 'gross', (float(row['grossPay'][1:]), line)
    except ValueError:
      self.increment_counter('warn','missing gross', 1)

  def reducer(self, key, values):
    topten = []
    for p in values :
      topten.append(p)
    #topten.sort()
    topten = topten[-100:]
    for p in topten:
      yield key, p

  combiner = reducer

if __name__ == '__main__':
  salarymax.run()

Overwriting top_salary.py


In [42]:
!python top_salary.py Baltimore_City_Employee_Salaries_2021.csv > top_salary_output_2021.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/top_salary.root.20250605.024139.757837
Running step 1 of 1...

Counters: 1
	warn
		missing salary=311

job output is in /tmp/top_salary.root.20250605.024139.757837/output
Streaming final output from /tmp/top_salary.root.20250605.024139.757837/output...
Removing temp directory /tmp/top_salary.root.20250605.024139.757837...


In [33]:
import csv

cols="lastName,middleInitial,firstName,jobClass,agencyName,agencyID,annualSalary,grossPay,hireDate,fiscalYear,ObjectId".split(",")

f = open('Baltimore_City_Employee_Salaries.csv', 'r')

line = f.readline()


reader = csv.reader([line])
values = next(reader)
row = dict(zip(cols, values))

print(row)
print((f"\n\nRow being processed: {row}\n\n").encode('utf-8'))

f.close()

{'lastName': '\ufeffAaron', 'middleInitial': 'D', 'firstName': 'Kareem', 'jobClass': 'UTILITIES INST REPAIR II', 'agencyName': 'Public Works - Water & Waste Water', 'agencyID': 'A50', 'annualSalary': '34115', 'grossPay': '44557.99', 'hireDate': '2018-08-27', 'fiscalYear': 'FY2021', 'ObjectId': '1'}
b"\n\nRow being processed: {'lastName': '\\ufeffAaron', 'middleInitial': 'D', 'firstName': 'Kareem', 'jobClass': 'UTILITIES INST REPAIR II', 'agencyName': 'Public Works - Water & Waste Water', 'agencyID': 'A50', 'annualSalary': '34115', 'grossPay': '44557.99', 'hireDate': '2018-08-27', 'fiscalYear': 'FY2021', 'ObjectId': '1'}\n\n"
